# Comparison Statistics between Cohorts

## Python Setup

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
from pandas import Series
from openpyxl import load_workbook
pd.options.display.max_columns = 100

## Load In Data

### Original Data

In [2]:
inca_grnts = pd.read_csv('../data/inca/inca_grants_details.csv', low_memory=False)
inca_pubs = pd.read_csv('../data/inca/inca_pub_details.csv', low_memory=False)
inca_orcid_responses = pd.read_csv('../output/researcher_info/researcher_info_ORCID_returns.csv')

In [3]:
inca_orcid_responses.head()

,name,inca_name,emails,INCA ID,Dimensions Researcher ID,Additional Researcher DIM ID to combine,Additional Researcher DIM ID to combine 2,URL 1,URL 2,URL 3,ORCID Return
0,ABDEL MAJID KHATIB,ABDEL MAJID KHATIB,majid.khatib@stlouis.inserm.fr,inca_729,ur.01207067257.39,NaN,NaN,https://inca.dimensions.ai/discover/publicatio...,NaN,NaN,NO
1,ADRIANA OLIVEIRA DOS SANTOS,ADRIANA OLIVEIRA DOS SANTOS,adriana.santos@curie.fr; santos.a.o@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
2,AFSHIN GANGI,AFSHIN GANGI,gangi@unistra.fr; gangi@rad6.u-strasbg.fr,inca_557,ur.01066731462.51,ur.012652700470.64,NaN,https://inca.dimensions.ai/discover/publicatio...,https://inca.dimensions.ai/discover/publicatio...,NaN,YES
3,AGNES CARAVATTI,AGNES CARAVATTI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
4,AGNES BERNET,AGNES BERNET,bernet@lyon.fnclcc.fr; agnes.bernet@lyon.unica...,inca_129,ur.0726141333.93,NaN,NaN,https://inca.dimensions.ai/discover/publicatio...,NaN,NaN,NO


In [4]:
# Clean Grants File
inca_grnts.columns = [x.lower().replace(' ', '_') for x in inca_grnts.columns]
inca_grnts.rename(columns={'inca_id': 'rsr_id'
                            , 'funding_amount_($)': 'funding_amount'
                            , 'dimensions_grant_id': 'grant_id'
                            , 'funder': 'funder_name'
                            , 'rcdc': 'rcdc_names'
                           }, inplace=True)
del inca_grnts['prenom_port'], inca_grnts['nom_port'], inca_grnts['title']
del inca_grnts['abstract'], inca_grnts['reference'], inca_grnts['organisme_port']
del inca_grnts['research_org_names'], inca_grnts['research_org_ids'], inca_grnts['for']
inca_grnts['rcdc_names'] = inca_grnts['rcdc_names'].replace(';', '; ', regex=True)

In [5]:
# Get INCA-funded Grants from file
inca_funders = ["French National Cancer Institute", "French Institute of Health and Medical Research"]
inca_funded_grnts = inca_grnts[(inca_grnts['funder_name'].isin(inca_funders))
                               & (pd.DatetimeIndex(inca_grnts['start_date']).year>=2007)
                               & (pd.DatetimeIndex(inca_grnts['start_date']).year<=2012)].reset_index(drop=True)

In [6]:
# Inca ORCID-responses: keep only when there was an ORCID response.
inca_orcid_responses = inca_orcid_responses[inca_orcid_responses['ORCID Return']=="YES"].reset_index(drop=True)
conf_ids = list(inca_orcid_responses['INCA ID'])

# Change Funder Name in INCa-Funded grants when they responded to ORCID
inca_funded_grnts['funder_name'] = (inca_funded_grnts['funder_name'] 
                                    + np.where(inca_funded_grnts['rsr_id'].isin(conf_ids), ' - ORCID Confirmed', ''))

In [7]:
# Clean Pubs File
inca_pubs.columns = [x.lower().replace(' ', '_') for x in inca_pubs.columns]
inca_pubs.rename(columns={'inca_id': 'rsr_id'
                          , 'dimensions_publication_id': 'pub_id'
                          , 'publication_year': 'date'
                          , 'rcdc': 'rcdc_names'
                          , 'times_cited': 'citations'
                         }, inplace=True)
del inca_pubs['prenom_port'], inca_pubs['nom_port'], inca_pubs['organisme_port']
del inca_pubs['dimensions_researcher_id'], inca_pubs['additional_researcher_dim_id_to_combine']
del inca_pubs['additional_researcher_dim_id_to_combine_2'], inca_pubs['orcid'], inca_pubs['title'], inca_pubs['issue']
del inca_pubs['pages'], inca_pubs['pubmed_id'], inca_pubs['volume'], inca_pubs['relative_citation_ratio']
del inca_pubs['altmetric'], inca_pubs['open_access'], inca_pubs['author_names'], inca_pubs['research_org_names']
del inca_pubs['research_org_ids'], inca_pubs['for'], inca_pubs['journal_id'], inca_pubs['journal_title']
del inca_pubs['publication_date']
inca_pubs['date'] = inca_pubs['date'].apply(str).replace('\.0', '', regex=True)
inca_pubs['date'] = inca_pubs['date'].apply(lambda x: np.nan if x=="nan" else x+"-01-01")
inca_pubs['rcdc_names'] = inca_pubs['rcdc_names'].replace(';', '; ', regex=True)
inca_pubs['citations'] = pd.to_numeric(inca_pubs['citations'])

### Counterfactual Data

In [8]:
funded_grnts = pd.read_csv('../data/counterfactual/counterfactual_funded_grants.csv')
grnts = pd.read_csv('../data/counterfactual/counterfactual_researcher_grants.csv')
pubs = pd.read_csv('../data/counterfactual/counterfactual_researcher_publications.csv')

In [9]:
# Restrict to Grants funded between 2007 and 2012
funded_grnts = funded_grnts[(pd.DatetimeIndex(funded_grnts['start_date']).year>=2007)
                            & (pd.DatetimeIndex(funded_grnts['start_date']).year<=2012)].reset_index(drop=True)

### Combining Data

In [10]:
funded_grnts = pd.concat([funded_grnts, inca_funded_grnts]).reset_index(drop = True)
grnts = pd.concat([grnts, inca_grnts]).reset_index(drop = True)
pubs = pd.concat([pubs, inca_pubs]).reset_index(drop = True)

### List of All Funders

In [11]:
funders = sorted(list(funded_grnts['funder_name'].unique()))
print(funders)

['Cancer Research UK', 'French Institute of Health and Medical Research', 'French Institute of Health and Medical Research - ORCID Confirmed', 'French National Cancer Institute', 'French National Cancer Institute - ORCID Confirmed', 'National Cancer Institute', 'National Health and Medical Research Council', 'Wellcome Trust']


## Cleaning Data

In [12]:
funded_grnts['start_date'] = pd.to_datetime(funded_grnts['start_date'])
funded_grnts['end_date'] = pd.to_datetime(funded_grnts['end_date'])
grnts['start_date'] = pd.to_datetime(grnts['start_date'])
grnts['end_date'] = pd.to_datetime(grnts['end_date'])
pubs['date'] = pd.to_datetime(pubs['date'])

In [13]:
funded_grnts['funding_len'] = (funded_grnts['end_date']-funded_grnts['start_date'])/timedelta(days=365)
grnts['funding_len'] = (grnts['end_date']-grnts['start_date'])/timedelta(days=365)

In [14]:
funded_grnts['nb_rsrs'] = funded_grnts.groupby('grant_id')['rsr_id'].transform('nunique')
funded_grnts['nb_rsrs'] = funded_grnts['nb_rsrs'].replace(0, np.nan)
grnts['nb_rsrs'] = grnts.groupby('grant_id')['rsr_id'].transform('nunique')
grnts['nb_rsrs'] = grnts['nb_rsrs'].replace(0, np.nan)
pubs['nb_rsrs'] = pubs.groupby('pub_id')['rsr_id'].transform('nunique')
pubs['nb_rsrs'] = pubs['nb_rsrs'].replace(0, np.nan)

## Funded Grants Statistics

### Number of Grants per Funder

In [15]:
funded_grnts.describe(include='all')

,end_date,funder_id,funder_name,funding_amount,grant_id,rcdc_codes,rcdc_names,rsr_id,start_date,funding_len,nb_rsrs
count,20973,20041,21020,1.875300e+04,21020,18580,19543,19759,21020,20973.000000,20386.000000
unique,1076,5,8,NaN,16894,7436,7486,12905,1176,NaN,NaN
top,2017-01-01 00:00:00,grid.48336.3a,National Cancer Institute,NaN,grant.2695966,503,Cancer,ur.01117731572.33,2009-01-01 00:00:00,NaN,NaN
freq,668,13701,13701,NaN,17,740,777,384,1566,NaN,NaN
first,2007-01-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-01-01 00:00:00,NaN,NaN
last,2024-10-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-12-31 00:00:00,NaN,NaN
mean,NaN,NaN,NaN,1.635865e+06,NaN,NaN,NaN,NaN,NaN,3.880751,1.706024
std,NaN,NaN,NaN,8.608239e+06,NaN,NaN,NaN,NaN,NaN,2.439551,1.629312
min,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.002740,1.000000
25%,NaN,NaN,NaN,2.723860e+05,NaN,NaN,NaN,NaN,NaN,2.002740,1.000000


In [16]:
print("Number of unique researchers funded by each agency:\n")
print(funded_grnts.groupby('funder_name')['rsr_id'].nunique())
nb_unique_rsrs = list(funded_grnts.groupby('funder_name')['rsr_id'].nunique())

Number of unique researchers funded by each agency:

funder_name
Cancer Research UK                                                   1370
French Institute of Health and Medical Research                       122
French Institute of Health and Medical Research - ORCID Confirmed      34
French National Cancer Institute                                      669
French National Cancer Institute - ORCID Confirmed                     95
National Cancer Institute                                            8450
National Health and Medical Research Council                         1944
Wellcome Trust                                                        328
Name: rsr_id, dtype: int64


In [17]:
print("Number of unique grants funded by each agency:\n")
print(funded_grnts.groupby('funder_name')['grant_id'].nunique())
nb_unique_grnts = list(funded_grnts.groupby('funder_name')['grant_id'].nunique())

Number of unique grants funded by each agency:

funder_name
Cancer Research UK                                                    2225
French Institute of Health and Medical Research                        114
French Institute of Health and Medical Research - ORCID Confirmed       35
French National Cancer Institute                                       730
French National Cancer Institute - ORCID Confirmed                     132
National Cancer Institute                                            12468
National Health and Medical Research Council                          1064
Wellcome Trust                                                         261
Name: grant_id, dtype: int64


### Number of Grants per Researcher

In [18]:
print("There are {} total counterfactual researchers.".format(funded_grnts['rsr_id'].nunique()))

There are 12905 total counterfactual researchers.


In [19]:
print("Number of grants from counterfactual agencies per researcher (as % of total):\n")
print(funded_grnts.groupby('rsr_id')['grant_id'].nunique().value_counts(normalize=True).head())

Number of grants from counterfactual agencies per researcher (as % of total):

1    0.731809
2    0.165285
3    0.052305
4    0.022627
5    0.013096
Name: grant_id, dtype: float64


In [20]:
print("By how many counterfactual agencies are the researchers funded?\n")
print(funded_grnts.groupby('rsr_id')['funder_name'].nunique().value_counts(normalize=True))

By how many counterfactual agencies are the researchers funded?

1    0.992019
2    0.007749
3    0.000155
4    0.000077
Name: funder_name, dtype: float64


In [21]:
print("How many grants does each agency give to its reserachers in the 5 focal years?")
for funder in funders:
    print("\n{}:".format(funder))
    temp = funded_grnts[funded_grnts['funder_name']==funder].groupby('rsr_id')['grant_id'].nunique().reset_index()
    print(temp['grant_id'].value_counts(normalize=True).head(3))
    print("(Total researchers: {})".format(temp.shape[0]))

How many grants does each agency give to its reserachers in the 5 focal years?

Cancer Research UK:
1    0.705109
2    0.161314
3    0.053285
Name: grant_id, dtype: float64
(Total researchers: 1370)

French Institute of Health and Medical Research:
1    0.983607
2    0.016393
Name: grant_id, dtype: float64
(Total researchers: 122)

French Institute of Health and Medical Research - ORCID Confirmed:
1    0.970588
2    0.029412
Name: grant_id, dtype: float64
(Total researchers: 34)

French National Cancer Institute:
1    0.814649
2    0.140508
3    0.026906
Name: grant_id, dtype: float64
(Total researchers: 669)

French National Cancer Institute - ORCID Confirmed:
1    0.726316
2    0.168421
3    0.073684
Name: grant_id, dtype: float64
(Total researchers: 95)

National Cancer Institute:
1    0.726036
2    0.173136
3    0.053609
Name: grant_id, dtype: float64
(Total researchers: 8450)

National Health and Medical Research Council:
1    0.745885
2    0.139918
3    0.057613
Name: grant_id, d

### Grant Characteristics

In [22]:
cols = list(funded_grnts)
cols.remove('rsr_id')
funded_grnts_info = funded_grnts[cols].drop_duplicates().reset_index(drop=True)

In [23]:
funded_grnts_info.head()

,end_date,funder_id,funder_name,funding_amount,grant_id,rcdc_codes,rcdc_names,start_date,funding_len,nb_rsrs
0,2018-11-30,grid.48336.3a,National Cancer Institute,3181288.0,grant.2482176,503; 546; 337; 316; 507,Cancer; Patient Safety; Bioengineering; Breast...,2012-12-31,5.917808,2.0
1,2015-12-27,grid.455095.8,French National Cancer Institute,727240.0,grant.7154902,526; 559; 344; 503,Genetics; Rare Diseases; Digestive Diseases; C...,2012-12-27,3.000000,1.0
2,2015-12-27,grid.455095.8,French National Cancer Institute,799578.0,grant.7154248,526; 559; 344; 503; 411,Genetics; Rare Diseases; Digestive Diseases; C...,2012-12-27,3.000000,1.0
3,2015-12-27,grid.455095.8,French National Cancer Institute,564589.0,grant.7154673,583; 503; 580,Vaccine Related; Cancer; Urologic Diseases,2012-12-27,3.000000,1.0
4,2015-12-27,grid.455095.8,French National Cancer Institute,615157.0,grant.7154822,559; 484; 313; 387; 503; 501; 568; 569,Rare Diseases; Stem Cell Research; Brain Cance...,2012-12-27,3.000000,NaN


In [24]:
funded_amt_avg = []
funded_amt_med = []
funded_len_avg = []
funded_len_med = []
nb_grnt_rsrs_avg = []
nb_grnt_rsrs_med = []
for funder in funders:
    print("\n{}:".format(funder))
    temp = funded_grnts_info[funded_grnts_info['funder_name']==funder]
    print(temp.describe())
    print("(Total grants: {})".format(funded_grnts_info[funded_grnts_info['funder_name']==funder].shape[0]))
    funded_amt_avg.append(temp['funding_amount'].mean())
    funded_amt_med.append(temp['funding_amount'].median())
    funded_len_avg.append(temp['funding_len'].mean())
    funded_len_med.append(temp['funding_len'].median())
    nb_grnt_rsrs_avg.append(temp['nb_rsrs'].mean())
    nb_grnt_rsrs_med.append(temp['nb_rsrs'].median())


Cancer Research UK:
       funding_amount  funding_len      nb_rsrs
count             0.0  2225.000000  2223.000000
mean              NaN     3.372391     1.016644
std               NaN     2.076851     0.127963
min               NaN     0.002740     1.000000
25%               NaN     1.997260     1.000000
50%               NaN     3.000000     1.000000
75%               NaN     5.000000     1.000000
max               NaN    13.008219     2.000000
(Total grants: 2225)

French Institute of Health and Medical Research:
       funding_amount  funding_len     nb_rsrs
count    1.140000e+02   113.000000  114.000000
mean     2.342411e+05     2.150152    1.114035
std      1.869081e+05     1.021059    0.345867
min      2.448100e+04     0.509589    1.000000
25%      1.279798e+05     1.408219    1.000000
50%      2.015395e+05     2.000000    1.000000
75%      3.020678e+05     2.997260    1.000000
max      1.259731e+06     5.002740    3.000000
(Total grants: 114)

French Institute of Health and M

In [25]:
funded_grnts_info['year'] = pd.DatetimeIndex(funded_grnts_info['start_date']).year
pd.crosstab(funded_grnts_info['funder_name'], funded_grnts_info['year'])

year,2007,2008,2009,2010,2011,2012
funder_name,,,,,,
Cancer Research UK,549,428,346,306,331,265
French Institute of Health and Medical Research,0,0,0,0,61,53
French Institute of Health and Medical Research - ORCID Confirmed,0,0,0,0,16,19
French National Cancer Institute,364,170,222,215,179,191
French National Cancer Institute - ORCID Confirmed,34,21,20,21,21,15
National Cancer Institute,1777,2592,2870,1780,1743,1706
National Health and Medical Research Council,141,198,198,106,210,211
Wellcome Trust,33,40,37,53,55,43


### RCDC Codes Analysis

In [26]:
funded_grnts_info['nb_rcdc'] = (funded_grnts_info['rcdc_names'].str.count(';')+1)

In [27]:
print("Average number of RCDC's per Grant:\n")
print(funded_grnts_info.groupby('funder_name')['nb_rcdc'].mean())

Average number of RCDC's per Grant:

funder_name
Cancer Research UK                                                   4.223325
French Institute of Health and Medical Research                      4.711712
French Institute of Health and Medical Research - ORCID Confirmed    5.117647
French National Cancer Institute                                     5.169324
French National Cancer Institute - ORCID Confirmed                   5.190840
National Cancer Institute                                            5.754231
National Health and Medical Research Council                         4.725564
Wellcome Trust                                                       4.498084
Name: nb_rcdc, dtype: float64


In [28]:
rcdc = funded_grnts_info[['grant_id', 'rcdc_names']]
rcdc = rcdc[rcdc['rcdc_names'].notnull()]

In [29]:
rcdc = pd.concat([Series(row['grant_id'], row['rcdc_names'].split('; '))
                  for _, row in rcdc.iterrows()]).reset_index()
rcdc.columns = ['rcdc_name', 'grant_id']

In [30]:
rcdc = pd.merge(funded_grnts_info[['funder_name', 'grant_id', 'year']], rcdc, how='left', on='grant_id')

In [31]:
rcdc.head()

,funder_name,grant_id,year,rcdc_name
0,National Cancer Institute,grant.2482176,2012,Cancer
1,National Cancer Institute,grant.2482176,2012,Patient Safety
2,National Cancer Institute,grant.2482176,2012,Bioengineering
3,National Cancer Institute,grant.2482176,2012,Breast Cancer
4,National Cancer Institute,grant.2482176,2012,Clinical Research


In [32]:
rcdc_1 = []
rcdc_2 = []
rcdc_3 = []
rcdc_4 = []
rcdc_5 = []

for funder in funders:
    print("\n{}:".format(funder))
    temp = rcdc[rcdc['funder_name']==funder]
    print(temp['rcdc_name'].value_counts().head())
    rcdc_1.append(temp['rcdc_name'].value_counts().index[0])
    rcdc_2.append(temp['rcdc_name'].value_counts().index[1])
    rcdc_3.append(temp['rcdc_name'].value_counts().index[2])
    rcdc_4.append(temp['rcdc_name'].value_counts().index[3])
    rcdc_5.append(temp['rcdc_name'].value_counts().index[4])


Cancer Research UK:
Cancer               1415
Clinical Research     628
Genetics              574
Rare Diseases         472
Biotechnology         316
Name: rcdc_name, dtype: int64

French Institute of Health and Medical Research:
Cancer               93
Rare Diseases        41
Genetics             36
Clinical Research    36
Biotechnology        31
Name: rcdc_name, dtype: int64

French Institute of Health and Medical Research - ORCID Confirmed:
Cancer               31
Genetics             16
Rare Diseases        15
Clinical Research    14
Biotechnology         9
Name: rcdc_name, dtype: int64

French National Cancer Institute:
Cancer               2500
Rare Diseases        1125
Genetics             1057
Clinical Research     954
Biotechnology         769
Name: rcdc_name, dtype: int64

French National Cancer Institute - ORCID Confirmed:
Cancer               268
Rare Diseases        119
Clinical Research    117
Genetics             114
Biotechnology         89
Name: rcdc_name, dtype: int6

## Prior and Subsequent Grants

In [33]:
def grant_groupby(grnts_mrg):
    cols = [col for col in grnts_mrg.columns if col[-2:]!="_2"]
    grnts_mrg['pre_flag'] = grnts_mrg['start_date']>grnts_mrg['start_date_2']
    grnts_mrg['post_flag'] = grnts_mrg['start_date']<grnts_mrg['start_date_2']
    for col in ['funding_amount', 'funding_len', 'nb_rsrs']:
        grnts_mrg['pre_'+col] = grnts_mrg[col+'_2']*grnts_mrg['pre_flag']
        grnts_mrg['post_'+col] = grnts_mrg[col+'_2']*grnts_mrg['post_flag']

    grnts_stats = grnts_mrg.groupby(['grant_id', 'rsr_id'])
    grnts_stats = pd.DataFrame({'pre_nb_grnts':grnts_stats['pre_flag'].sum()
                                , 'pre_fund_amt':grnts_stats['pre_funding_amount'].sum()
                                , 'pre_avg_fund_len':grnts_stats['pre_funding_len'].mean()
                                , 'pre_avg_team_size':grnts_stats['pre_nb_rsrs'].mean()
                                , 'post_nb_grnts':grnts_stats['post_flag'].sum()
                                , 'post_fund_amt':grnts_stats['post_funding_amount'].sum()
                                , 'post_avg_fund_len':grnts_stats['post_funding_len'].mean()
                                , 'post_avg_team_size':grnts_stats['post_nb_rsrs'].mean()
                               }).reset_index()
    temp = grnts_mrg[cols].copy().drop_duplicates()
    grnts_stats = pd.merge(temp, grnts_stats, how='left', on=['grant_id', 'rsr_id'])
    
    return grnts_stats

In [34]:
def grant_stats(grnts_stats):
    
    print("Pre-Funding Grant Statistics:")
    pre_avg_fund_len_avg = []
    pre_avg_fund_len_med = []
    pre_avg_team_size_avg = []
    pre_avg_team_size_med = []
    pre_fund_amt_avg = []
    pre_fund_amt_med = []
    pre_nb_grnts_avg = []
    pre_nb_grnts_med = []
    cols = [col for col in grnts_stats.columns if col[:4]=="pre_"]
    for funder in funders:
        print("\n{}:".format(funder))
        temp = grnts_stats[grnts_stats['funder_name']==funder]
        print(temp[cols].describe())
        pre_avg_fund_len_avg.append(temp['pre_avg_fund_len'].mean())
        pre_avg_fund_len_med.append(temp['pre_avg_fund_len'].median())
        pre_avg_team_size_avg.append(temp['pre_avg_team_size'].mean())
        pre_avg_team_size_med.append(temp['pre_avg_team_size'].median())
        pre_fund_amt_avg.append(temp['pre_fund_amt'].mean())
        pre_fund_amt_med.append(temp['pre_fund_amt'].median())
        pre_nb_grnts_avg.append(temp['pre_nb_grnts'].mean())
        pre_nb_grnts_med.append(temp['pre_nb_grnts'].median())
        
    print("\n")
    
    print("Post-Funding Grant Statistics:")
    post_avg_fund_len_avg = []
    post_avg_fund_len_med = []
    post_avg_team_size_avg = []
    post_avg_team_size_med = []
    post_fund_amt_avg = []
    post_fund_amt_med = []
    post_nb_grnts_avg = []
    post_nb_grnts_med = []
    cols = [col for col in grnts_stats.columns if col[:5]=="post_"]
    for funder in funders:
        print("\n{}:".format(funder))
        temp = grnts_stats[grnts_stats['funder_name']==funder]
        print(temp[cols].describe())
        post_avg_fund_len_avg.append(temp['post_avg_fund_len'].mean())
        post_avg_fund_len_med.append(temp['post_avg_fund_len'].median())
        post_avg_team_size_avg.append(temp['post_avg_team_size'].mean())
        post_avg_team_size_med.append(temp['post_avg_team_size'].median())
        post_fund_amt_avg.append(temp['post_fund_amt'].mean())
        post_fund_amt_med.append(temp['post_fund_amt'].median())
        post_nb_grnts_avg.append(temp['post_nb_grnts'].mean())
        post_nb_grnts_med.append(temp['post_nb_grnts'].median())

    return pre_avg_fund_len_avg, pre_avg_fund_len_med, pre_avg_team_size_avg, pre_avg_team_size_med, pre_fund_amt_avg, pre_fund_amt_med, pre_nb_grnts_avg, pre_nb_grnts_med, post_avg_fund_len_avg, post_avg_fund_len_med, post_avg_team_size_avg, post_avg_team_size_med, post_fund_amt_avg, post_fund_amt_med, post_nb_grnts_avg, post_nb_grnts_med

### For all Grants

In [35]:
# grnts_mrg = pd.merge(funded_grnts, grnts, how='left', on='rsr_id', suffixes=('', '_2'))
# grnts_stats_all = grant_groupby(grnts_mrg)

In [36]:
# grnts_stats_all.shape

In [37]:
# grnts_stats_all.describe(include='all')

In [38]:
# grant_stats(grnts_stats_all)

### Within 5 years of the funded grant

In [39]:
grnts_mrg = pd.merge(funded_grnts, grnts, how='left', on='rsr_id', suffixes=('', '_2'))
grnts_mrg = grnts_mrg[abs(grnts_mrg['start_date']-grnts_mrg['start_date_2'])/timedelta(days=365)<=5]
grnts_mrg = grnts_mrg.reset_index(drop=True)
grnts_stats_5y = grant_groupby(grnts_mrg)

In [40]:
grnts_stats_5y.shape

(19759, 19)

In [41]:
grnts_stats_5y.describe(include='all')

,end_date,funder_id,funder_name,funding_amount,grant_id,rcdc_codes,rcdc_names,rsr_id,start_date,funding_len,nb_rsrs,post_avg_fund_len,post_avg_team_size,post_fund_amt,post_nb_grnts,pre_avg_fund_len,pre_avg_team_size,pre_fund_amt,pre_nb_grnts
count,19715,18780,19759,1.749400e+04,19759,17709,18672,19759,19759,19715.000000,19759.000000,19736.000000,19759.000000,1.975900e+04,19759.000000,19736.000000,19759.000000,1.975900e+04,19759.00000
unique,1067,5,8,NaN,16260,7154,7432,12905,1168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2017-01-01 00:00:00,grid.48336.3a,National Cancer Institute,NaN,grant.2695966,503,Cancer,ur.01117731572.33,2009-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,667,13084,13084,NaN,17,695,732,384,1490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,2007-01-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2024-10-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-12-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,1.641870e+06,NaN,NaN,NaN,NaN,NaN,3.991859,1.724328,0.904749,0.596292,2.201047e+06,1.754694,0.825679,0.455139,1.585434e+06,1.62574
std,NaN,NaN,NaN,4.403675e+06,NaN,NaN,NaN,NaN,NaN,2.438066,1.650495,1.049920,0.962810,8.253946e+06,4.571717,1.088619,0.798642,5.446857e+06,4.35548
min,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.002740,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.00000
25%,NaN,NaN,NaN,2.947962e+05,NaN,NaN,NaN,NaN,NaN,2.002740,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.00000


In [42]:
pre_avg_fund_len_avg, pre_avg_fund_len_med, pre_avg_team_size_avg, pre_avg_team_size_med, pre_fund_amt_avg, pre_fund_amt_med, pre_nb_grnts_avg, pre_nb_grnts_med, post_avg_fund_len_avg, post_avg_fund_len_med, post_avg_team_size_avg, post_avg_team_size_med, post_fund_amt_avg, post_fund_amt_med, post_nb_grnts_avg, post_nb_grnts_med = grant_stats(grnts_stats_5y)

Pre-Funding Grant Statistics:

Cancer Research UK:
       pre_avg_fund_len  pre_avg_team_size  pre_fund_amt  pre_nb_grnts
count       2260.000000        2260.000000  2.260000e+03   2260.000000
mean           0.996603           0.546492  9.956313e+05      2.298230
std            1.035040           0.891362  2.824911e+06      3.028198
min            0.000000           0.000000  0.000000e+00      0.000000
25%            0.000000           0.000000  0.000000e+00      0.000000
50%            0.885073           0.333333  0.000000e+00      1.000000
75%            1.708333           0.666667  5.200978e+05      3.000000
max            7.153973          12.000000  3.148749e+07     19.000000

French Institute of Health and Medical Research:
       pre_avg_fund_len  pre_avg_team_size  pre_fund_amt  pre_nb_grnts
count        123.000000         124.000000  1.240000e+02    124.000000
mean           0.373357           0.204397  2.722241e+05      0.564516
std            0.681347           0.386033  7.8

## Prior and Subsequent Publications

In [43]:
def pub_groupby(pubs_mrg):
    cols = [col for col in pubs_mrg.columns if col[-2:]!="_2"]
    pubs_mrg['pre_flag'] = pubs_mrg['start_date']>pubs_mrg['date_2']
    pubs_mrg['post_flag'] = pubs_mrg['start_date']<pubs_mrg['date_2']
    for col in ['citations']:
        pubs_mrg['pre_'+col] = pubs_mrg[col+'_2']*pubs_mrg['pre_flag']
        pubs_mrg['post_'+col] = pubs_mrg[col+'_2']*pubs_mrg['post_flag']

    pubs_stats = pubs_mrg.groupby(['grant_id', 'rsr_id'])
    pubs_stats = pd.DataFrame({'pre_nb_pubs':pubs_stats['pre_flag'].sum()
                                , 'pre_citations':pubs_stats['pre_citations'].sum()
                                , 'post_nb_pubs':pubs_stats['post_flag'].sum()
                                , 'post_citations':pubs_stats['post_citations'].sum()
                               }).reset_index()
    temp = pubs_mrg[cols].copy().drop_duplicates()
    pubs_stats = pd.merge(temp, pubs_stats, how='left', on=['grant_id', 'rsr_id'])
    
    return pubs_stats

In [44]:
def pub_stats(pubs_stats):
    
    print("Pre-Funding Publication Statistics:")
    pre_citations_avg = []
    pre_nb_pubs_avg = []
    pre_citations_med = []
    pre_nb_pubs_med = []
    cols = [col for col in pubs_stats.columns if col[:4]=="pre_"]
    for funder in funders:
        print("\n{}:".format(funder))
        temp = pubs_stats[pubs_stats['funder_name']==funder]
        print(temp[cols].describe())
        pre_citations_avg.append(temp['pre_citations'].mean())
        pre_citations_med.append(temp['pre_citations'].median())
        pre_nb_pubs_avg.append(temp['pre_nb_pubs'].mean())
        pre_nb_pubs_med.append(temp['pre_nb_pubs'].median())
    
    print("\n")
    
    print("Post-Funding Publication Statistics:")
    post_citations_avg = []
    post_citations_med = []
    post_nb_pubs_avg = []
    post_nb_pubs_med = []
    cols = [col for col in pubs_stats.columns if col[:5]=="post_"]
    for funder in funders:
        print("\n{}:".format(funder))
        temp = pubs_stats[pubs_stats['funder_name']==funder]
        print(temp[cols].describe())
        post_citations_avg.append(temp['post_citations'].mean())
        post_citations_med.append(temp['post_citations'].median())
        post_nb_pubs_avg.append(temp['post_nb_pubs'].mean())
        post_nb_pubs_med.append(temp['post_nb_pubs'].median())
        
    return pre_citations_avg, pre_nb_pubs_avg, pre_citations_med, pre_nb_pubs_med, post_citations_avg, post_citations_med, post_nb_pubs_avg, post_nb_pubs_med

### For all Publications

In [45]:
# pubs_mrg = pd.merge(funded_grnts, pubs, how='left', on='rsr_id')
# pubs_mrg.columns = [col+"_2" if (col not in list(funded_grnts))&(col[-2:]!="_2") else col for col in pubs_mrg.columns]
# pubs_stats_all = pub_groupby(pubs_mrg)

In [46]:
# pubs_stats_all.shape

In [47]:
# pubs_stats_all.describe(include='all')

In [48]:
# pre_citations_avg, pre_nb_pubs_avg, pre_citations_med, pre_nb_pubs_med, post_citations_avg, post_citations_med, post_nb_pubs_avg, post_nb_pubs_med = pub_stats(pubs_stats_all)

### Within 5 years of the funded grant

In [49]:
pubs_mrg = pd.merge(funded_grnts, pubs, how='left', on='rsr_id', suffixes=('', '_2'))
pubs_mrg.columns = [col+"_2" if (col not in list(funded_grnts))&(col[-2:]!="_2") else col for col in pubs_mrg.columns]
pubs_mrg = pubs_mrg[abs(pubs_mrg['start_date']-pubs_mrg['date_2'])/timedelta(days=365)<=5]
pubs_mrg = pubs_mrg.reset_index(drop=True)
pubs_stats_5y = pub_groupby(pubs_mrg)

In [50]:
pubs_stats_5y.shape

(15919, 15)

In [51]:
pubs_stats_5y.describe(include='all')

,end_date,funder_id,funder_name,funding_amount,grant_id,rcdc_codes,rcdc_names,rsr_id,start_date,funding_len,nb_rsrs,post_citations,post_nb_pubs,pre_citations,pre_nb_pubs
count,15894,14987,15919,1.445400e+04,15919,14432,15349,15919,15919,15894.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000
unique,944,5,8,NaN,13304,6366,6657,10456,1120,NaN,NaN,NaN,NaN,NaN,NaN
top,2017-01-01 00:00:00,grid.48336.3a,National Cancer Institute,NaN,grant.2695966,503,Cancer,ur.0634201432.39,2009-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
freq,538,10969,10969,NaN,17,502,534,127,1194,NaN,NaN,NaN,NaN,NaN,NaN
first,2007-01-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
last,2023-05-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-12-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,1.599928e+06,NaN,NaN,NaN,NaN,NaN,3.978853,1.725171,1494.556756,40.909982,1856.970538,29.734531
std,NaN,NaN,NaN,4.030727e+06,NaN,NaN,NaN,NaN,NaN,2.305065,1.646338,2738.810501,46.657556,3159.450382,34.969784
min,NaN,NaN,NaN,3.900000e+01,NaN,NaN,NaN,NaN,NaN,0.002740,1.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,3.150122e+05,NaN,NaN,NaN,NaN,NaN,2.002740,1.000000,227.000000,12.000000,318.500000,9.000000


In [52]:
pre_citations_avg, pre_nb_pubs_avg, pre_citations_med, pre_nb_pubs_med, post_citations_avg, post_citations_med, post_nb_pubs_avg, post_nb_pubs_med = pub_stats(pubs_stats_5y)

Pre-Funding Publication Statistics:

Cancer Research UK:
       pre_citations  pre_nb_pubs
count    1462.000000  1462.000000
mean     2095.625171    29.878933
std      2728.844275    31.750003
min         0.000000     0.000000
25%       418.750000    10.000000
50%      1080.500000    21.000000
75%      2543.250000    40.000000
max     21056.000000   393.000000

French Institute of Health and Medical Research:
       pre_citations  pre_nb_pubs
count     116.000000   116.000000
mean      794.896552    28.413793
std      1336.182390    37.870734
min         0.000000     0.000000
25%        88.250000     7.000000
50%       350.000000    18.500000
75%       849.250000    35.250000
max      9027.000000   277.000000

French Institute of Health and Medical Research - ORCID Confirmed:
       pre_citations  pre_nb_pubs
count      35.000000    35.000000
mean      518.228571    22.714286
std       719.033505    23.951632
min         0.000000     0.000000
25%        73.500000     8.000000
50%      

## Creating Output Table

Table with funders as columns and:
- 1st RCDC Code
- 2nd RCDC Code
- 3rd RCDC Code
- Mean number of previous grants
- Median number of previous grants
- Mean amount of previous grants
- Median number of previous grants
- Mean length of previous grants
- Median length of previous grants

In [53]:
df = pd.DataFrame({'rcdc_1':rcdc_1
                    , 'rcdc_2':rcdc_2
                    , 'rcdc_3':rcdc_3
                    , 'rcdc_4':rcdc_4
                    , 'rcdc_5':rcdc_5                   
                    , 'nb_unique_rsrs': nb_unique_rsrs
                    , 'nb_unique_grnts': nb_unique_grnts
                    , 'funded_amt_avg': funded_amt_avg
                    , 'funded_amt_med': funded_amt_med
                    , 'funded_len_avg': funded_len_avg
                    , 'funded_len_med': funded_len_med
                    , 'nb_grnt_rsrs_avg': nb_grnt_rsrs_avg
                    , 'nb_grnt_rsrs_med': nb_grnt_rsrs_med
                    , 'pre_avg_fund_len_avg': pre_avg_fund_len_avg
                    , 'pre_avg_fund_len_med': pre_avg_fund_len_med
                    , 'pre_avg_team_size_avg': pre_avg_team_size_avg
                    , 'pre_avg_team_size_med': pre_avg_team_size_med
                    , 'pre_fund_amt_avg': pre_fund_amt_avg
                    , 'pre_fund_amt_med': pre_fund_amt_med
                    , 'pre_nb_grnts_avg': pre_nb_grnts_avg
                    , 'pre_nb_grnts_med': pre_nb_grnts_med
                    , 'post_avg_fund_len_avg': post_avg_fund_len_avg
                    , 'post_avg_fund_len_med': post_avg_fund_len_med
                    , 'post_avg_team_size_avg': post_avg_team_size_avg
                    , 'post_avg_team_size_med': post_avg_team_size_med
                    , 'post_fund_amt_avg': post_fund_amt_avg
                    , 'post_fund_amt_med': post_fund_amt_med
                    , 'post_nb_grnts_avg': post_nb_grnts_avg
                    , 'post_nb_grnts_med': post_nb_grnts_med
                    , 'pre_citations_avg': pre_citations_avg
                    , 'pre_nb_pubs_avg': pre_nb_pubs_avg
                    , 'pre_citations_med': pre_citations_med
                    , 'pre_nb_pubs_med': pre_nb_pubs_med
                    , 'post_citations_avg': post_citations_avg
                    , 'post_citations_med': post_citations_med
                    , 'post_nb_pubs_avg': post_nb_pubs_avg
                    , 'post_nb_pubs_med': post_nb_pubs_med
                  }, index=funders).transpose()
df

,Cancer Research UK,French Institute of Health and Medical Research,French Institute of Health and Medical Research - ORCID Confirmed,French National Cancer Institute,French National Cancer Institute - ORCID Confirmed,National Cancer Institute,National Health and Medical Research Council,Wellcome Trust
funded_amt_avg,NaN,234241,232874,557125,464459,1.85636e+06,609689,797714
funded_amt_med,NaN,201540,235320,417676,471555,732061,433722,325304
funded_len_avg,3.37239,2.15015,2.08211,2.73308,2.65841,4.25496,2.8267,3.22012
funded_len_med,3,2,1.99726,3,3,3.9589,3,3.08493
nb_grnt_rsrs_avg,1.01664,1.11404,1.08571,1.38386,1.49242,1.10404,2.69925,1.57088
nb_grnt_rsrs_med,1,1,1,1,1,1,2,1
nb_unique_grnts,2225,114,35,730,132,12468,1064,261
nb_unique_rsrs,1370,122,34,669,95,8450,1944,328
post_avg_fund_len_avg,0.962587,0.197712,0.170896,0.501654,0.689976,0.945569,0.847795,0.847452
post_avg_fund_len_med,0.833333,0,0,0,0,0.500913,0.81868,0.500587


In [54]:
# Export to Excel
ls = !ls ../output/
if 'comparison_statistics.xlsx' in ls:
    book = load_workbook('../output/comparison_statistics.xlsx')
    writer = pd.ExcelWriter('../output/comparison_statistics.xlsx', engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df.to_excel(writer, "RAW")
    writer.save()
else:
    df.to_excel('../output/comparison_statistics.xlsx', sheet_name = 'RAW')

## Sandbox